In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
from IPython.display import display, clear_output
import ipywidgets as widgets


In [16]:
data = pd.read_csv("diverse_synthetic_customer_data.csv")
data['Preferences'] = data['Preferences'].fillna('')
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['Preferences'])

numerical_columns = ['Age', 'ProductQuality', 'ServiceQuality', 'PurchaseFrequency', 'SatisfactionScore']
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[numerical_columns])

engine = create_engine('sqlite:///user_profiles.db', echo=False)
data.to_sql('profiles', con=engine, if_exists='replace', index=False)


1000

In [17]:
def classify_user_type(row):
    if row['PurchaseFrequency'] > 15 and row['SatisfactionScore'] >= 4:
        return 'Loyal'
    elif row['SatisfactionScore'] < 3:
        return 'Dissatisfied'
    elif row['Age'] < 25:
        return 'Young'
    else:
        return 'General'

data['UserType'] = data.apply(classify_user_type, axis=1)

def store_recommendations(user_id, recommendations):
    with engine.connect() as conn:
        recommendations['UserID'] = user_id
        recommendations['Timestamp'] = pd.Timestamp.now()
        recommendations.to_sql('recommendations_history', con=conn, if_exists='append', index=False)


In [18]:
def versatile_recommend(user_id, top_n=5):
    if user_id not in data['CustomerID'].values:
        return f"User with CustomerID {user_id} not found.", pd.DataFrame()

    user_index = data[data['CustomerID'] == user_id].index[0]
    user_type = data.loc[user_index, 'UserType']

    preference_similarities = cosine_similarity(tfidf_matrix[user_index], tfidf_matrix).flatten()
    numerical_similarities = cosine_similarity([data_scaled[user_index]], data_scaled).flatten()
    combined_similarities = 0.6 * preference_similarities + 0.4 * numerical_similarities

    data['SimilarityScore'] = combined_similarities

    similar_users = data[(data['UserType'] == user_type) & (data.index != user_index)]
    similar_users = similar_users.sort_values(by='SimilarityScore', ascending=False)

    recommendations = similar_users.head(top_n)
    store_recommendations(user_id, recommendations[['CustomerID', 'Preferences']])

    return "Recommendations generated successfully.", recommendations[['CustomerID', 'Preferences', 'ProductQuality', 'ServiceQuality']]


In [20]:
def handle_new_customer_ui():
    new_customer_output = widgets.Output()  # Define the output widget here

    def submit_new_customer(_):
        new_customer_output.clear_output()

        customer_id = new_customer_id_input.value
        preferences = preferences_input.value
        age = age_input.value
        purchase_frequency = purchase_frequency_input.value
        satisfaction_score = satisfaction_score_input.value
        product_quality = product_quality_input.value
        service_quality = service_quality_input.value

        new_data = pd.DataFrame({
            'CustomerID': [customer_id],
            'Age': [age],
            'Preferences': [preferences],
            'ProductQuality': [product_quality],
            'ServiceQuality': [service_quality],
            'PurchaseFrequency': [purchase_frequency],
            'SatisfactionScore': [satisfaction_score]
        })

        new_data['UserType'] = new_data.apply(classify_user_type, axis=1)
        new_tfidf_matrix = tfidf.transform(new_data['Preferences'])
        new_scaled_data = scaler.transform(new_data[numerical_columns])

        preference_similarities = cosine_similarity(new_tfidf_matrix, tfidf_matrix).flatten()
        numerical_similarities = cosine_similarity(new_scaled_data, data_scaled).flatten()
        combined_similarities = 0.6 * preference_similarities + 0.4 * numerical_similarities

        data['SimilarityScore'] = combined_similarities

        similar_users = data[data['UserType'] == new_data['UserType'][0]]
        similar_users = similar_users.sort_values(by='SimilarityScore', ascending=False)

        recommendations = similar_users.head(5)
        store_recommendations(customer_id, recommendations[['CustomerID', 'Preferences']])

        with new_customer_output:
            print("Recommendations generated successfully.")
            display(recommendations[['CustomerID', 'Preferences', 'ProductQuality', 'ServiceQuality']])

    def back_to_main_ui(_):
        clear_output(wait=True)
        main_ui()

    new_customer_id_input = widgets.Text(description="Customer ID:")
    preferences_input = widgets.Text(description="Preferences:")
    age_input = widgets.IntText(description="Age:")
    purchase_frequency_input = widgets.IntText(description="Purchase Frequency:")
    satisfaction_score_input = widgets.IntSlider(description="Satisfaction:", min=1, max=5, value=3)
    product_quality_input = widgets.IntSlider(description="Product Quality:", min=1, max=5, value=3)
    service_quality_input = widgets.IntSlider(description="Service Quality:", min=1, max=5, value=3)

    submit_button = widgets.Button(description="Submit")
    back_button = widgets.Button(description="Back to Main Menu")

    submit_button.on_click(submit_new_customer)
    back_button.on_click(back_to_main_ui)

    display(
        widgets.VBox([
            widgets.HTML("<h3 style='color:blue;'>New Customer Registration</h3>"),
            new_customer_id_input, preferences_input, age_input, purchase_frequency_input,
            satisfaction_score_input, product_quality_input, service_quality_input,
            submit_button, new_customer_output, back_button
        ])
    )


In [21]:
def handle_existing_customer_ui():
    def submit_existing_customer(_):
        nonlocal existing_customer_output
        existing_customer_output.clear_output()

        customer_id = existing_customer_id_input.value
        top_n = top_n_input.value

        message, recommendations = versatile_recommend(customer_id, top_n)
        with existing_customer_output:
            print(message)
            display(recommendations)

    def back_to_main_ui(_):
        clear_output(wait=True)
        main_ui()

    existing_customer_id_input = widgets.Text(description="Customer ID:")
    top_n_input = widgets.IntText(description="Top N Recommendations:", value=5)
    submit_button = widgets.Button(description="Submit")
    existing_customer_output = widgets.Output()
    back_button = widgets.Button(description="Back to Main Menu")

    submit_button.on_click(submit_existing_customer)
    back_button.on_click(back_to_main_ui)

    display(
        widgets.VBox([
            widgets.HTML("<h3 style='color:green;'>Existing Customer Recommendations</h3>"),
            existing_customer_id_input, top_n_input,
            submit_button, existing_customer_output, back_button
        ])
    )


In [22]:
def handle_update_preferences_ui():
    def submit_update_preferences(_):
        nonlocal update_preferences_output
        update_preferences_output.clear_output()

        customer_id = update_customer_id_input.value
        new_preferences = new_preferences_input.value

        if customer_id in data['CustomerID'].values:
            data.loc[data['CustomerID'] == customer_id, 'Preferences'] = new_preferences
            global tfidf_matrix
            tfidf_matrix = tfidf.fit_transform(data['Preferences'])
            with engine.connect() as conn:
                data.to_sql('profiles', con=conn, if_exists='replace', index=False)

            with update_preferences_output:
                print(f"Preferences updated for Customer ID: {customer_id}")
        else:
            with update_preferences_output:
                print(f"Customer ID {customer_id} not found.")

    def back_to_main_ui(_):
        clear_output(wait=True)
        main_ui()

    update_customer_id_input = widgets.Text(description="Customer ID:")
    new_preferences_input = widgets.Text(description="New Preferences:")
    submit_button = widgets.Button(description="Submit")
    update_preferences_output = widgets.Output()
    back_button = widgets.Button(description="Back to Main Menu")

    submit_button.on_click(submit_update_preferences)
    back_button.on_click(back_to_main_ui)

    display(
        widgets.VBox([
            widgets.HTML("<h3 style='color:red;'>Update Preferences</h3>"),
            update_customer_id_input, new_preferences_input,
            submit_button, update_preferences_output, back_button
        ])
    )


In [23]:
def main_ui():
    def handle_choice(change):
        clear_output(wait=True)
        if change['new'] == 'New Customer':
            handle_new_customer_ui()
        elif change['new'] == 'Existing Customer':
            handle_existing_customer_ui()
        elif change['new'] == 'Update Preferences':
            handle_update_preferences_ui()

    user_choice = widgets.ToggleButtons(
        options=['New Customer', 'Existing Customer', 'Update Preferences'],
        description='Choose:',
        button_style='info'
    )
    user_choice.observe(handle_choice, names='value')
    display(user_choice)

if __name__ == "__main__":
    main_ui()


<ipython-input-17-c571b6636fe2>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['UserID'] = user_id
<ipython-input-17-c571b6636fe2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['Timestamp'] = pd.Timestamp.now()
